In [4]:
from Chess_Version_6 import Game, Custom_Game, Player, Board, square, Piece, Pawn, Queen, Rook, King, Knight, Bishop
import numpy as np
import math
import copy
import time
import sys

In [5]:
def FEN_update(info):
    
    move,move_counts,board,currplayer,notakes,genmoves,castles = info
    
    d = {'w':'b','b':'w'}
    
    #Move is a tuple with the piece, current position,the position to move it to, and a tag for an en passant
    #Board is the list of lists representation of the board
    if len(move[2]) == 2:
        x,y = move[2]
        xcurr,ycurr = move[1]
    elif len(move[2]) == 3:
        x,y,prom = move[2]
        xcurr,ycurr = move[1]
    
    ep = abs(ycurr-y)
    p = move[0]
    move_counts[p][0] += 1
    move_counts[p][1] = move[2]
    b2 = board[ycurr].copy()
    b2[xcurr] = '*'
    
    board[ycurr] = b2
    
    if board[y][x] != '*':
        notakes = 0
    else:
        notakes += 1
    
    a2 = board[y].copy()
    a2[x] = p
    board[y] = a2
    
    fen  = ''
    
    cnt = 0
    for row in board[::-1]:
        for sq in row:

            if sq == '*':
                cnt += 1
            else:
                if cnt > 0:
                    fen += str(cnt)
                    cnt = 0
                fen += sq[0]
        if cnt > 0 or cnt == 8:
            fen += str(cnt)
        cnt = 0
        fen +='/'
    epstatus = ' - '
    if p[0].lower() == 'p':
        if ep==2:
            epstatus = ' '+'abcdefgh'[x]+str(y+1)+' '
    elif p[0].lower() == 'k':
        if currplayer == 'w':
            castles[0] = ''
        elif currplayer == 'b':
            castles[1] = ''
    elif p[0].lower() == 'r':
        if move_counts[p] != 0:
            side = p[1]
            if side.lower() == side:
                if side in castles[1]:
                    castles[1] = castles[1].replace(side,'')
            elif side.lower() != side:
                if side in castles[0]:
                    castles[0] = castles[0].replace(side,'')
        
    if currplayer == 'b':
        genmoves += 1
    fen += ' ' + d[currplayer] +' '+ castles[0] + castles[1] + epstatus + str(notakes)+' '+str(genmoves)

    return (fen,move_counts,board,d[currplayer],notakes,genmoves,castles)
        
def possible_game(info):
    
    fen,move_counts,board,currplay,notakes,genmoves,castles = info
    
    p_dict = {'p': Pawn,'n':Knight,'b':Bishop,'r':Rook,'q':Queen,'k':King}
    
    cust_game = Custom_Game()
    
    mct,pos = move_counts['k'] 
    k = King(cust_game.ChessBoard.squares[pos[1]][pos[0]],'Black','k',cust_game.ChessBoard)
    mct,pos = move_counts['K'] 
    K = King(cust_game.ChessBoard.squares[pos[1]][pos[0]],'White','K',cust_game.ChessBoard)
    
    cust_game.ChessBoard.add_pieces([k,K])
    
    kings = {'White':K,'Black':k}
    
    p_list = []
    
    for p in move_counts:
        if p.lower() != 'k':
        
            if p[0].lower() == p[0]:
                col = 'Black'

            elif p[0].lower() != p[0]:
                col = 'White'

            mct,pos = move_counts[p]
            
            p2 = p_dict[p[0].lower()](cust_game.ChessBoard.squares[pos[1]][pos[0]],col,p,cust_game.ChessBoard,kings)
            p2.move_count = mct
            p_list.append(p2)
        
    cust_game.ChessBoard.add_pieces(p_list)
    b = cust_game.ChessBoard.update_board()
    b = Player('Butt','Black',cust_game.ChessBoard)
    w = Player('Deriere','White',cust_game.ChessBoard)
    d = {'b':'w','w':'b'}
    playerdict = {'w':w,'b':b}
    cust_game.WhitePlayer = w
    cust_game.BlackPlayer = b
    
    cust_game.ChessBoard.heat_setup()
    cust_game.ChessBoard.en_passant_setup()
    cust_game.ChessBoard.players = {'White':w,'Black':b}
    cust_game.ChessBoard.current_player = playerdict[d[currplay]]
    cust_game.ChessBoard.notakes = notakes
    cust_game.ChessBoard.move_count = genmoves
    cust_game.ChessBoard.FEN = fen
    cust_game.ChessBoard.wcastle_rights = castles[0]
    cust_game.ChessBoard.bcastle_rights = castles[1]
    cust_game.ChessBoard.heat_setup()
    cust_game.ChessBoard.en_passant_setup()
    
    p = name = b = w = playerdict = p_list = kings = fen = move_counts=board=currplay=notakes=genmoves=castles = None 
    
    return cust_game

In [25]:
def FEN_to_Game(fen):
    cg = Custom_Game()
    lstf = fen.split(" ")
    cast = lstf[-4]
    mcnt_bord= int(lstf[-2])
    ep = lstf[-3]
    sqdic = {'a':0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
    if ep[0].isalpha():
        epcoor = (sqdic[ep[0]],int(ep[1]))
    currpl = lstf[-5]
    notake = int(lstf[-1])
    pieces = lstf[0].split('/')[::-1]
    pieces = [p for p in pieces if p]
    pdict = {'p':Pawn,'k':King,'q':Queen,'b':Bishop,'r':Rook,'n':Knight}
    pcnt = {'p':0,'P':0,'n':0,'N':0,'b':0,'B':0,'r':0,'R':0,'q':0,'Q':0}
    for i in range(len(pieces)):
        y = i
        x = 0
        mcnt = 1
        for j in range(len(pieces[i])):
            if pieces[i][j].isnumeric():
                x += int(pieces[i][j])

            elif pieces[i][j].isalpha():
                if pieces[i][j].lower() == 'k':
                    pos = (x,y)

                    if pieces[i][j] == pieces[i][j].lower():
                        color = 'Black'
                        for ch in cast:
                            if ch == 'k' or ch == 'q':
                                mcnt = 0
                        k = King(cg.ChessBoard.squares[pos[1]][pos[0]],color,'k',cg.ChessBoard)
                        cg.ChessBoard.squares[pos[1]][pos[0]].occupied = [color,'k']
                    elif pieces[i][j].upper() == pieces[i][j]:
                        color = 'White'
                        for ch in cast:
                            if ch == 'K' or ch == 'Q':
                                mcnt = 0
                        K = King(cg.ChessBoard.squares[pos[1]][pos[0]],color,'K',cg.ChessBoard)
                        cg.ChessBoard.squares[pos[1]][pos[0]].occupied = [color,'K']
                x += 1
                
    cg.ChessBoard.add_pieces([K,k])
    kings = {'White':K,'Black':k}
    plst = []
    for i in range(len(pieces)):
        y = i
        x = 0
        for j in range(len(pieces[i])):
            if pieces[i][j].isnumeric():
                x += int(pieces[i][j])

            elif pieces[i][j].isalpha():
                if pieces[i][j].lower() != 'k':
                    pos = (x,y)
                    mcnt = 1
                    name = pieces[i][j]+str(pcnt[pieces[i][j]])
                    pcnt[pieces[i][j]] += 1
                    if pieces[i][j] == pieces[i][j].lower():
                        color = 'Black'
                    else:
                        color = 'White'
                    if pieces[i][j].lower() == 'p':
                            if (pos[1] == 1 and pieces[i][j] == 'P') or (pos[1] == 6 and pieces[i][j] == 'p'):
                                mcnt = 0

                    elif pieces[i][j].lower() == 'r':
                        for ch in cast:
                            Qside = (ch == 'Q' and pos == (0,0))
                            qside = (ch == 'q' and pos == (0,7))
                            kside = (ch == 'k' and pos == (7,7))
                            Kside = (ch == 'K' and pos == (7,0))
                            if Qside or qside or kside or Kside:
                                mcnt = 0

                    piece = pdict[pieces[i][j].lower()](cg.ChessBoard.squares[pos[1]][pos[0]],color,name,cg.ChessBoard,kings)
                    cg.ChessBoard.squares[pos[1]][pos[0]].occupied = [color,name]
                    piece.move_count = mcnt
                    plst.append(piece)
                x += 1
    cg.ChessBoard.add_pieces(plst)
    b = cg.ChessBoard.update_board()
    b = Player('Butt','Black',cg.ChessBoard)
    w = Player('Deriere','White',cg.ChessBoard)
    d = {'b':'w','w':'b'}
    playerdict = {'w':w,'b':b}
    cg.WhitePlayer = w
    cg.BlackPlayer = b
    
    cg.ChessBoard.heat_setup()
    cg.ChessBoard.en_passant_setup()
    cg.ChessBoard.players = {'White':w,'Black':b}
    cg.ChessBoard.current_player = playerdict[currpl]
    cg.ChessBoard.notakes = int(notake)
    cg.ChessBoard.move_count = mcnt_bord
    cg.ChessBoard.FEN = fen
    wcas = [i for i in cast if i.isalpha() and i.upper() == i]
    bcas = [j for j in cast if j.isalpha() and j.lower() == j]
    wstr = ''
    bstr = ''
    for wc in wcas:
        wstr += wc
    for bc in bcas:
        bstr += bc
    cg.ChessBoard.wcastle_rights = wstr
    cg.ChessBoard.bcastle_rights = bstr
    cg.ChessBoard.heat_setup()
    cg.ChessBoard.en_passant_setup()
    if ep[0].isalpha():
        col,occ = cg.ChessBoard.squares[epcoor[1]][epcoor[0]].occupied
        cg.ChessBoard.en_passant[col][occ] = True
    cg.ChessBoard.kings = kings
    return cg

In [26]:
"r6r/1b2k1bq/8/8/7B/8/8/R3K2R b KQ - 3 2".split(' ')

['r6r/1b2k1bq/8/8/7B/8/8/R3K2R', 'b', 'KQ', '-', '3', '2']

In [27]:
game = FEN_to_Game("8/8/8/2k5/2pP4/8/B7/4K3 b - d3 0 3")
game.ChessBoard.players['White'].possible_moves()
game.ChessBoard.players['Black'].possible_moves()

game.ChessBoard.current_player.possible_moves()

{'k': [(3, 4), (1, 5), (1, 4), (3, 3), (2, 5), (1, 3), (3, 5)], 'p0': [(3, 2)]}

In [28]:
game.ChessBoard.update_board()[::-1]

[['*', '*', '*', '*', '*', '*', '*', '*'],
 ['*', '*', '*', '*', '*', '*', '*', '*'],
 ['*', '*', '*', '*', '*', '*', '*', '*'],
 ['*', '*', 'k', '*', '*', '*', '*', '*'],
 ['*', '*', 'p0', 'P0', '*', '*', '*', '*'],
 ['*', '*', '*', '*', '*', '*', '*', '*'],
 ['B0', '*', '*', '*', '*', '*', '*', '*'],
 ['*', '*', '*', '*', 'K', '*', '*', '*']]

In [29]:
fens = []
numpos = []
depths = []
cnt = 0
with open('evalpos') as f:
    reed = f.read()
    for i in reed.split('\n'):          
        b = i.lstrip()
        c = b.rstrip()
        if len(c) > 1:  
            if c[1].isalpha():
                if c[1] == 'n':
                    val = int(c.split(':')[1][:-1])
                    numpos.append(val)
                elif c[1] == 'f':
                    fen = c.split(':')[1][1:-1]
                    fens.append(fen)    
                elif c[1] == 'd':
                    dep = int(c.split(':')[1][:-1])
                    depths.append(dep)

In [30]:
testdict = {i:j for i,j in zip(fens,numpos)}
for i in testdict.keys():
    try:
        g = FEN_to_Game(i)
        g.ChessBoard.players['White'].possible_moves()
        g.ChessBoard.players['Black'].possible_moves()
        mvtots = g.ChessBoard.current_player.possible_moves()
        total = 0
        for key in mvtots.keys():
            total += len(mvtots[key])
        if total == testdict[i]:
            print(f'Yippee, no worries for this case: {i}')
        else:
            print(f'My game gave a total number of {total}, it should be {testdict[i]} for {i}')
    except:
        print(f'issue creating the game for {i}')

Yippee, no worries for this case: r6r/1b2k1bq/8/8/7B/8/8/R3K2R b KQ - 3 2
Yippee, no worries for this case: 8/8/8/2k5/2pP4/8/B7/4K3 b - d3 0 3
Yippee, no worries for this case: r1bqkbnr/pppppppp/n7/8/8/P7/1PPPPPPP/RNBQKBNR w KQkq - 2 2
Yippee, no worries for this case: r3k2r/p1pp1pb1/bn2Qnp1/2qPN3/1p2P3/2N5/PPPBBPPP/R3K2R b KQkq - 3 2
Yippee, no worries for this case: 2kr3r/p1ppqpb1/bn2Qnp1/3PN3/1p2P3/2N5/PPPBBPPP/R3K2R b KQ - 3 2
Yippee, no worries for this case: rnb2k1r/pp1Pbppp/2p5/q7/2B5/8/PPPQNnPP/RNB1K2R w KQ - 3 9
Yippee, no worries for this case: 2r5/3pk3/8/2P5/8/2K5/8/8 w - - 5 4
My game gave a total number of 44, it should be 62379 for rnbq1k1r/pp1Pbppp/2p5/8/2B5/8/PPP1NnPP/RNBQK2R w KQ - 1 8
My game gave a total number of 46, it should be 89890 for r4rk1/1pp1qppp/p1np1n2/2b1p1B1/2B1P1b1/P1NP1N2/1PP1QPPP/R4RK1 w - - 0 10
My game gave a total number of 18, it should be 1134888 for 3k4/3p4/8/K1P4r/8/8/8/8 b - - 0 1
My game gave a total number of 13, it should be 1015133 for 8/8

In [31]:
depths

[1, 1, 1, 1, 1, 1, 1, 3, 3, 6, 6, 6, 6, 6, 4, 4, 6, 5, 6, 6, 6, 7, 4]

In [10]:
#numpos

In [38]:
g = FEN_to_Game('K1k5/8/P7/8/8/8/8/8 w - - 0 1')
depth = 1
total = 0
g.ChessBoard.pieces['White']['P0'].kings

{'White': <Chess_Version_6.King at 0x23986de3ca0>,
 'Black': <Chess_Version_6.King at 0x23986de3ee0>}

In [ ]:
total = 0
cnt3 = 0
fend = {}
class Node:
    
    def __init__(self,game,depth,topdep,ct,pt = None):
        
        self.g = game
        self.depth = depth
        self.visited = 0
        self.td = topdep
        self.tot = ct
        ch = []
        self.pt = pt
        mvpos = self.g.ChessBoard.current_player.possible_moves()
        col = self.g.ChessBoard.current_player.color
        ctot = 0
        for key in mvpos.keys():
            if mvpos[key]:
                ctot += len(mvpos[key])
        
        self.currtot = ctot
        self.tot += ctot
        
        #print(f',{ctot}')
        global total
        total += ctot
        print(self.depth)
        if self.depth < topdep:
            
            for key in mvpos.keys():
                for i in mvpos[key]:
                    if i:
                        c = copy.deepcopy(self.g)
                        c.ChessBoard.pieces[col][key].move_to((i))
                        ch.append(Node(c,depth+1,topdep,self.tot,self))
                        
        self.children = ch
head = Node(g,0,6,0)
print(head.tot)

0
1
2
3
4
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6


6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5


6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6


6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
4
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6


5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6


6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6


In [ ]:
total